In [26]:
import os
os.environ['TRANSFORMERS_CACHE'] = os.path.join(os.getcwd(), 'transformers')
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [27]:
import pfrl
from textrl import TextRLEnv, TextRLActor, train_agent_with_evaluation
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import logging
import sys

In [28]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

checkpoint = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)
# , torch_dtype="auto", device_map="auto"
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.eval()
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [29]:
sentiment_path_1 = "cardiffnlp/twitter-roberta-base-sentiment" # 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
sentiment_path_2 = 'nlptown/bert-base-multilingual-uncased-sentiment'
sentiment_1 = pipeline('sentiment-analysis',model=sentiment_path_1,tokenizer=sentiment_path_1, top_k=None)
sentiment_2 = pipeline('sentiment-analysis',model=sentiment_path_2,tokenizer=sentiment_path_2, top_k=None)

In [30]:
sentiment_1("\xa0an (anonymous) (anonymous) (anonymous) (anonymous (anonymous) (anonymous)")

[[{'label': 'LABEL_1', 'score': 0.8726766705513},
  {'label': 'LABEL_0', 'score': 0.0813116580247879},
  {'label': 'LABEL_2', 'score': 0.046011634171009064}]]

In [31]:
sentiment_2("\xa0an (anonymous) (anonymous) (anonymous) (anonymous (anonymous) (anonymous)")

[[{'label': '1 star', 'score': 0.5140755772590637},
  {'label': '2 stars', 'score': 0.22503311932086945},
  {'label': '3 stars', 'score': 0.15606173872947693},
  {'label': '4 stars', 'score': 0.0636562630534172},
  {'label': '5 stars', 'score': 0.041173290461301804}]]

In [32]:
def return_score_nlptown(sentence, percent=1):
    scores = sentiment_2(sentence)[0]
    req_score = 0
    for dict_inner in scores:
        if dict_inner['label'] == '1 star':
            req_score += percent * dict_inner['score']
        if dict_inner['label'] == '2 star':
            req_score += (1 - percent) * dict_inner['score']

    return req_score

def return_score_cardiff_roberta_xlm(sentence):
    scores = sentiment_1(sentence)[0]
    req_score = 0
    for dict_inner in scores:
        if dict_inner['label'] == 'negative':
            req_score += 10 * dict_inner['score']
        if dict_inner['label'] == 'positive':
            req_score -= 10 * dict_inner['score']
        if dict_inner['label'] == 'neutral':
            req_score -= 5 * dict_inner['score']

    return req_score

In [33]:
from nltk.translate import bleu_score
from nltk.tokenize import word_tokenize

class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish):  # predicted will be the list of predicted token
        rewards = []
        for i in predicted_list:
            if finish or len(i) > self.env_max_length:
                predicted_text = tokenizer.convert_tokens_to_string(i)
                # reward = 10 * bleu_score.sentence_bleu(references=[word_tokenize(input_item['output'])], hypothesis=word_tokenize(predicted_text))
                # rewards.append(reward)
                reward = return_score_cardiff_roberta_xlm(input_item['input']+predicted_text)
                reward = min(reward, return_score_nlptown(input_item['input']+predicted_text))
                rewards.append(reward)
                # calculate reward score base on predicted_list
            else:
                rewards.append(0)
        return rewards

In [34]:
import pandas as pd

df = pd.read_csv('newsCompiled.csv')
df['ol'] = df.apply(lambda x: {'input': x['Input'], 'output': x['Output']}, axis=1)
observation_list = df['ol'].tolist()

In [35]:
observaton_list = [{"input": "Dogecoin is "}]
env = MyRLEnv(
    model, tokenizer, 
    observation_input=observation_list, 
    max_length=25, 
    compare_sample=5
)
actor = TextRLActor(
    env, model, tokenizer, 
    act_deterministically=False, 
    optimizer='adamw',
    temperature=0.8,
    top_k=100,
    top_p=0.85
)
# act_deterministically=False,
# temperature=1.0,
# top_k=0,
# top_p=1.0,
# repetition_penalty=2

In [36]:
agent = actor.agent_ppo(
    update_interval=2, 
    minibatch_size=2, 
    epochs=10,
    lr=3e-4
)

test_phrase = observation_list[0]['input']
print(test_phrase)
actor.predict({'input': test_phrase})

Fed official says weak data


[' from China will lead to lower interest rates.\n\n\nChina has been increasing its interest rate for the last few months,',
 " would be 'deadly for the government'\n\n\nThe government has announced a $10bn budget for 2015-16",
 ' is to blame for lack of progress\n\n\nMOSCOW (Sputnik) — Russian Foreign Ministry spokesman Sergei',
 ' and sluggish economic growth have helped make the move to buy Chinese bonds.\n\n\n"The market is not ready to give',
 " can be used to limit Russia's nuclear program\n\n\nThe country is building a new nuclear reactor to replace a damaged one"]

In [12]:
train_agent_with_evaluation(
    agent,
    env,
    steps=100,
    eval_n_steps=None,
    eval_n_episodes=5,
    eval_interval=25,
    outdir='bloom—test-3',
)

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/pfrl/agents/ppo.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)


outdir:bloom—test-3 step:27 episode:0 R:0
statistics:[('average_value', -0.7015761), ('average_entropy', 1.4301476), ('average_value_loss', 0.1687923411326483), ('average_policy_loss', -0.13913709436356989), ('n_updates', 130), ('explained_variance', 0.7841575385543729)]
evaluation episode 0 length:27 R:0
evaluation episode 1 length:27 R:0
evaluation episode 2 length:27 R:0
evaluation episode 3 length:27 R:0
evaluation episode 4 length:27 R:0
The best score is updated -3.4028235e+38 -> 0.0
Saved the agent to bloom—test-3/best
outdir:bloom—test-3 step:54 episode:1 R:0
statistics:[('average_value', -0.118922256), ('average_entropy', 1.0440704), ('average_value_loss', 0.16484971854835748), ('average_policy_loss', -0.08675848134048252), ('n_updates', 270), ('explained_variance', -81.05598398428091)]
evaluation episode 0 length:27 R:0
evaluation episode 1 length:27 R:0
evaluation episode 2 length:27 R:0
evaluation episode 3 length:27 R:0
evaluation episode 4 length:27 R:0
outdir:bloom—test-

(<textrl.actor.TextPPO at 0x7fcc72910eb8>,
 [{'average_value': -0.7015761,
   'average_entropy': 1.4301476,
   'average_value_loss': 0.1687923411326483,
   'average_policy_loss': -0.13913709436356989,
   'n_updates': 130,
   'explained_variance': 0.7841575385543729,
   'eval_score': 0.0},
  {'average_value': -0.118922256,
   'average_entropy': 1.0440704,
   'average_value_loss': 0.16484971854835748,
   'average_policy_loss': -0.08675848134048252,
   'n_updates': 270,
   'explained_variance': -81.05598398428091,
   'eval_score': 0.0},
  {'average_value': 0.16733886,
   'average_entropy': 0.8581701,
   'average_value_loss': 0.05740455741994083,
   'average_policy_loss': -0.0681951494831589,
   'n_updates': 400,
   'explained_variance': -4.441601598863723,
   'eval_score': 0.0},
  {'average_value': 0.16058105,
   'average_entropy': 0.76002496,
   'average_value_loss': 0.01793369373306632,
   'average_policy_loss': -0.03996437291009352,
   'n_updates': 500,
   'explained_variance': 0.01306

In [13]:
agent.load('./bloom—test-3/best')
print(observation_list[0])
actor.predict(observation_list[0])

{'input': 'Fed official says weak data', 'output': 'caused by weather, should not slow taper'}


[' has been a factor\n\n\nThere has been a strong rally in support of a proposal to place a $1 trillion tax',
 ' has been found\n\n\nGDP has been about 30% more in recent years\n\n\nGDP has been about',
 ' has been a factor\n\n\nIf this has been a type of type of type of type of type which has been a',
 " has been used to blame Russia\n\n\n'We may have been misled about this type of research which has been conducted in",
 ' has been a factor in some of the worst financial crisis in recent memory\n\n\nMore than a third of about 5 million']